### Introduction

This lab environment allows access to the following models:

- A selection of Amazon Bedrock models.
- The Flan T5 Base model.

#### Installing, Upgrading, and Importing Dependencies

The following cells use the `pip` Python package manager to install `boto3` and `pandas` before importing them.

In [ ]:
!pip install --upgrade boto3 pandas

In [36]:
import boto3
import json
from IPython.display import JSON

#### Bedrock

The following is an example of invoking the Titan Premier model using the Amazon Bedrock API. 

In [37]:


def invoke_bedrock_model(model_id, prompt, options={}):
    bedrock = boto3.client("bedrock-runtime", region_name="us-east-1")

    body = {
        "inputText": prompt, 
    }
    
    response = bedrock.invoke_model(
        modelId=model_id,
        accept="application/json",
        contentType="application/json",
        body=json.dumps(body)
    )
    
    return response['body'].read()



In [ ]:

response = invoke_bedrock_model("amazon.titan-text-premier-v1:0", "Explain the main differences between supervised and unsupervised learning.")
print(response)
print(json.loads(response))
JSON(json.loads(response), expanded=True)

*Notes*:

- You can use a different model by supplying a different `model_id`.
- Different models use different body formats. You will need to modify the function above to use a different model.
- Options such as `top_k` and `temperature` can be added to the `body`.

#### SageMaker Flan-T5-Base

The following is an example of invoking the [flan-t5-base](https://huggingface.co/google/flan-t5-base) model using the Amazon SageMaker AI API.

In [39]:
def invoke_sagemaker_endpoint(endpoint_name, request_data):

    sagemaker_runtime = boto3.client("sagemaker-runtime", region_name="us-east-1")
    payload = json.dumps(request_data)

    response = sagemaker_runtime.invoke_endpoint(
        EndpointName=endpoint_name,
        ContentType="application/json",  # The MIME type of your request payload
        Body=payload
    )
    
    return response["Body"].read().decode("utf-8")

In [ ]:
endpoint_name = "qa-flan-t5-base"
response = invoke_sagemaker_endpoint(endpoint_name=endpoint_name, request_data={"inputs": "Explain the main differences between supervised and unsupervised learning."})
print(response)
JSON(json.loads(str(response)))

*Notes*:

- The Flan T5 Base model has been deployed on a `ml.g5.2xlarge` instance.

### Accessing the Billsum Dataset

The following cells download a copy of the [FiscalNote/billsum](https://huggingface.co/datasets/FiscalNote/billsum) dataset, extracts it, and loads it into a Pandas data frame.

In [ ]:
!curl -sL "https://assets.labs.platform.qa.com/stepstone-ai-academy-third-hackathon/billsum.zip" -o ./billsum.zip
!unzip billsum.zip

In [ ]:
import pandas as pd
df = pd.read_parquet("data/train-00000-of-00001.parquet")
print(df.head())